# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [171]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import corr
from pyspark.ml.feature import StringIndexer,VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [172]:
spark = SparkSession.builder.appName("logistic_regression").getOrCreate()

In [173]:
from pyspark.ml.classification import LogisticRegression

In [174]:
data = spark.read.csv("customer_churn.csv",inferSchema = True, header = True)
data = data.withColumn("string_date", data["Onboard_date"].cast("string"))

In [175]:
def string_indexer(dataframe,input_column, output_column):
    indexer = StringIndexer(inputCol = input_column,outputCol = output_column)
    return indexer.fit(dataframe).transform(dataframe)


In [176]:
data = string_indexer(data,"Location","location_index")
data = string_indexer(data,"Company","company_index")
data = string_indexer(data, "string_date","date_index")

In [177]:
temp_data = data.select("Age","Total_Purchase","Years","Num_sites","date_index","location_index","company_index","Churn")
temp_data.columns

['Age',
 'Total_Purchase',
 'Years',
 'Num_sites',
 'date_index',
 'location_index',
 'company_index',
 'Churn']

In [178]:
def vector_assembler(dataframe,input_columns,output_column):
    assembler = VectorAssembler(inputCols = input_columns, outputCol = output_column)
    return assembler.transform(dataframe)


In [179]:
temp_data = vector_assembler(temp_data,['Age',
 'Total_Purchase',
 'Years',
 'Num_sites',
 'date_index',
 'location_index',
 'company_index'],"features")

In [180]:
final_data = temp_data.select("features","Churn")
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [181]:
lr = LogisticRegression(labelCol = "Churn")
log_model = lr.fit(train_data)

In [182]:
results = log_model.evaluate(test_data)
results.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,4....|    0|[4.56775719846447...|[0.98972544573748...|       0.0|
|[25.0,9672.03,5.4...|    0|[3.75375863794182...|[0.97710685817924...|       0.0|
|[26.0,8787.39,5.4...|    1|[1.04564782712014...|[0.73993828717096...|       0.0|
|[26.0,8939.61,4.5...|    0|[5.74512079836148...|[0.99681184864497...|       0.0|
|[27.0,8628.8,5.3,...|    0|[6.11168519214311...|[0.99778809147587...|       0.0|
|[28.0,8670.98,3.9...|    0|[7.22769845441114...|[0.99927433666832...|       0.0|
|[28.0,11128.95,5....|    0|[3.89461642613300...|[0.98005473061734...|       0.0|
|[29.0,13240.01,4....|    0|[6.32879761135676...|[0.99821899986585...|       0.0|
|[29.0,13255.05,4....|    0|[4.74969259378168...|[0.99141990003586...|       0.0|
|[30.0,6744.87,5

In [183]:
binary_evaluator = BinaryClassificationEvaluator(rawPredictionCol = "prediction",labelCol = "Churn",metricName = "areaUnderROC")
binary_evaluator.evaluate(results.predictions)

0.7591202835105275

In [184]:
multi_class_evaluator = MulticlassClassificationEvaluator(predictionCol = "prediction",labelCol = "Churn",metricName = "f1")
multi_class_evaluator.evaluate(results.predictions)

0.8935337476513947

In [187]:
new_customers = spark.read.csv("new_customers.csv",inferSchema = True, header = True)
new_customers = new_customers.withColumn("string_date", new_customers["Onboard_date"].cast("string"))
new_customers = string_indexer(new_customers,"Location","location_index")
new_customers = string_indexer(new_customers,"Company","company_index")
new_customers = string_indexer(new_customers, "string_date","date_index")
new_customers = vector_assembler(new_customers,['Age',
 'Total_Purchase',
 'Years',
 'Num_Sites',
 'date_index',
 'location_index',
 'company_index'], "features")

In [194]:
final_model = lr.fit(final_data)
results = final_model.transform(new_customers)
results.select("Company","Prediction").show()

+----------------+----------+
|         Company|Prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [195]:
multi_class_evaluator.evaluate(results)

IllegalArgumentException: 'Field "Churn" does not exist.\nAvailable fields: Names, Age, Total_Purchase, Account_Manager, Years, Num_Sites, Onboard_date, Location, Company, string_date, location_index, company_index, date_index, features, rawPrediction, probability, prediction'